# Import Packages

In [1]:
import utils
from itertools import cycle
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
sns.set(style="ticks")
%config IPCompleter.greedy = True


# Import Data

In [2]:
df_calendar, df_sales, df_prices = utils.import_m5_data()

In [3]:
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


# Compute Forecasts

In [4]:
def forecast_snaive(df_sales, fh=28):
    fc_dict = {'id': df_sales['id']}
    day = 1
    for i in cycle(reversed(range(1, 8))):
        fc_dict[f'F{day}'] = df_sales.iloc[:, -i]
        if day == fh:
            break
        day += 1
    return pd.DataFrame(fc_dict)

In [5]:
forecasts = forecast_snaive(df_sales)

forecasts

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1,1,1,3,0,1,1,1,1,...,0,1,1,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,4,1,0,1,3,7,2,4,1,...,3,7,2,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,0,1,1,2,2,2,4,0,1,...,2,2,4,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,2,0,1,0,0,1,0,2,0,...,0,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,0,0,1,0,3,1,3,0,0,...,3,1,3,0,0,1,0,3,1,3


# Submit Predictions

In [6]:
df_submission = forecasts.append(
    forecasts.replace(to_replace='validation', value='evaluation', regex=True))
filename = utils.get_m5_root_dir() + '/data/submissions/snaive.csv'
df_submission.to_csv(filename, index=False)
df_submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1,1,1,3,0,1,1,1,1,...,0,1,1,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,4,1,0,1,3,7,2,4,1,...,3,7,2,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,0,1,1,2,2,2,4,0,1,...,2,2,4,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_evaluation,2,0,1,0,0,1,0,2,0,...,0,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_evaluation,0,0,1,0,3,1,3,0,0,...,3,1,3,0,0,1,0,3,1,3


In [7]:
# Submit via Kaggle API or with website https://www.kaggle.com/c/m5-forecasting-accuracy/submissions
# !kaggle competitions submit -c m5-forecasting-accuracy -f data/submissions/snaive.csv -m "sNaive"

# Compute MSE on last available 28 days 

In [8]:
df_sales_train = df_sales.iloc[:,:-28]
forecasts = forecast_snaive(df_sales_train)

In [9]:
y_pred_flat = forecasts.drop('id', axis=1).values.flatten()
y_true_flat = df_sales.iloc[:,-28:].values.flatten()

In [10]:
print('MSE of predicted values: %0.3f' % mean_squared_error(y_pred_flat, y_true_flat))

MSE of predicted values: 7.615
